In [1]:
import acl
import numpy as np
from PIL import Image
from acl_sample import Sample
import struct
from constant import NPY_BYTE
from postprocessing import postprocess_boxes
import cv2

In [2]:
device_id = 0
input_size = 416
model_path = "./model/yolov3_tf.om"
img_dict = {"path": "./data/dog.jpg", "dtype": np.uint8}

In [3]:
sample = Sample(device_id,
                model_path,
                input_size,
                input_size)

[Sample] init resource stage:
[Sample] init resource stage success
[Model] class Model init resource stage:
model output size 3
output  0
model output dims ({'name': 'yolov3/yolov3_head/feature_map_1:0', 'dimCount': 4, 'dims': [1, 13, 13, 255]}, 0)
model output datatype 0
output  1
model output dims ({'name': 'yolov3/yolov3_head/feature_map_2:0', 'dimCount': 4, 'dims': [1, 26, 26, 255]}, 0)
model output datatype 0
output  2
model output dims ({'name': 'yolov3/yolov3_head/feature_map_3:0', 'dimCount': 4, 'dims': [1, 52, 52, 255]}, 0)
model output datatype 0
[Model] create model output dataset:
[Model] create model output dataset success
[Model] class Model init resource stage success


In [10]:
# %timeit -n 100 -r 10 
model_output = sample.forward(img_dict)

[Sample] width:768 height:576
[Sample] image:./data/dog.jpg
[Sample] img_buffer_size:171363
[Dvpp] vpc decode stage:
[Dvpp] vpc decode stage success
[Dvpp] vpc resize stage:
[Dvpp] vpc resize stage success
dvpp_output_size 259584
[Model] create model input dataset:
[Model] create model input dataset success
[Model] execute stage:
[Model] execute stage success


In [11]:
def get_model_output_by_index(model_output, i, num_classes = 80):
    temp_output_buf = acl.mdl.get_dataset_buffer(model_output, i)

    infer_output_ptr = acl.get_data_buffer_addr(temp_output_buf)
    infer_output_size = acl.get_data_buffer_size(temp_output_buf)
    print("infer_output_size", infer_output_size)
    
#     output_host, _ = acl.rt.malloc_host(infer_output_size)
#     acl.rt.memcpy(output_host, infer_output_size, infer_output_ptr,
#                           infer_output_size, ACL_MEMCPY_DEVICE_TO_HOST)
    
    result = acl.util.ptr_to_numpy(infer_output_ptr, (infer_output_size,), NPY_BYTE)
    return np.array(struct.unpack(f"{infer_output_size//4}f", bytearray(result)), dtype=np.float32)
# .reshape(-1, 255)

In [12]:
pred_sbbox = get_model_output_by_index(model_output, 0)
pred_mbbox = get_model_output_by_index(model_output, 1)
pred_lbbox = get_model_output_by_index(model_output, 2)

infer_output_size 172380
infer_output_size 689520
infer_output_size 2758080


In [16]:
# with open("out.txt", "r") as fd:
#     lines = fd.readlines()

In [17]:
# tf_bbox = np.array([float(e) for e in lines[0].split()])
# .reshape(-1, 255)

In [ ]:
Image.open(img_dict["path"]).size

In [38]:
postprocess_boxes(pred_bbox, (576, 768), input_size, 0.3)

array([], shape=(0, 6), dtype=float64)